# Statewide Stops and Routes Dataframes

In [21]:
import gcsfs
import pandas as pd

BUCKET = "gs://gtfs-data-test/schedule/2021-04-11T00:00:00+00:00"
fs = gcsfs.GCSFileSystem(project='cal-itp-data-infra')
status = pd.read_csv(fs.open(f'{BUCKET}/status.csv'))
stops = fs.glob(f'{BUCKET}/*/*/processed/stops.txt')
agency = fs.glob(f'{BUCKET}/*/*/processed/agency.txt')
routes = fs.glob(f'{BUCKET}/*/*/processed/routes.txt')

In [23]:
def read_gtfs_data(path, itp_id, url_number):
    df = pd.read_csv(fs.open(path))
    return df.assign(
        calitp_itp_id = itp_id,
        calitp_url_number = url_number
    )

agencies = []
for ii, row in status[status.status == "success"].iterrows():
    agency_path = f"{BUCKET}/{row['itp_id']}/{row['url_number']}"
    
    if not fs.exists(f"{agency_path}/processed"):
        print("agency not processed: %s" % agency_path)
        continue

    stops = read_gtfs_data(f"{agency_path}/processed/stops.txt", row["itp_id"], row["url_number"])
    agency = read_gtfs_data(f"{agency_path}/processed/agency.txt", row["itp_id"], row["url_number"])
    routes = read_gtfs_data(f"{agency_path}/processed/routes.txt", row["itp_id"], row["url_number"])
    
    agencies.append(dict(stops = stops, agency = agency, routes = routes))
    

agency not processed: gs://gtfs-data-test/schedule/2021-04-11T00:00:00+00:00/232/0


In [24]:
all_stops = pd.concat([entry["stops"] for entry in agencies])
all_agency = pd.concat([entry["agency"] for entry in agencies])
all_routes = pd.concat([entry["routes"] for entry in agencies])

In [32]:
all_agency.shape

(210, 10)

In [33]:
all_stops.shape

(101408, 16)

In [27]:
all_stops.to_csv('~/Downloads/all_stops.csv')

In [29]:
all_routes.to_csv('~/Downloads/all_routes.csv')